In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import csv
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import sys, os
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from data_manager import *
from metadata import *

Setting environment for Local Macbook Pro


In [3]:
from annotation_utilities import *
from registration_utilities import *
from conversion import *

In [4]:
brain_annotation_file_map = \
{
'm920LGN': os.path.join(ROOT_DIR, 'LGN3Datlas', 'm920LGN.json'),
'm919LGN': os.path.join(ROOT_DIR, 'LGN3Datlas', 'm919LGN.json'),
}

In [5]:
XY_PIXEL_SIZE_UM = 0.46

In [6]:
import json

In [ ]:
# for stack in brain_annotation_file_map.keys():
for stack in ['m920LGN', 'm919LGN']:
    
    print stack

    input_file = brain_annotation_file_map[stack]
    
    with open(input_file, 'r') as f:
        data = json.load(f)

    contours = defaultdict(list)
    for name, cnts in data.iteritems():
        for cnt in cnts:
            if 'xy' in cnt:
                contours[name].append(np.column_stack([np.array(cnt['xy']) * XY_PIXEL_SIZE_UM, cnt['z']*np.ones((len(cnt['xy']),))]))
    contours.default_factory = None

    contour_colors = dict(zip(contours.keys(), np.array(random_colors(len(contours)))/255.))
    
    #############################
    
    plt.figure(figsize=(10, 10));
    for name, cnts in contours.iteritems():    
        for cnt in cnts:
            plt.plot(cnt[:, 0], cnt[:, 1], c=contour_colors[name]);

    plt.xlabel('x');
    plt.ylabel('y');
    plt.axis('equal');
    # plt.legend();

    ##############################

    plt.figure(figsize=(10, 10));
    for name, cnts in contours.iteritems():    
        for cnt in cnts:
            plt.plot(cnt[:, 2], cnt[:, 1], c=contour_colors[name]);

    plt.xlabel('x');
    plt.ylabel('y');
    plt.axis('equal');
    # plt.legend();

    out_resolution = '10.0um'
    out_resolution_um = convert_resolution_string_to_um(resolution=out_resolution)

    contours_orientationCorrected = {convert_to_left_name(name_u): [cnt[:, [2,1,0]]*[1,-1,1]
                                     for cnt in cnts3d] 
                           for name_u, cnts3d in contours.iteritems()}
#     contours_orientationCorrected = {convert_to_left_name(name_u): [cnt
#                                      for cnt in cnts3d] 
#                            for name_u, cnts3d in contours.iteritems()}

    contours_atlasResol = {name: [cnt / out_resolution_um
                                    for cnt in cnts3d if len(cnt) > 3] 
                           for name, cnts3d in contours_orientationCorrected.iteritems()}

    
    # Convert contours to volumes

    valid_level = .5

#     surround_distance_um = 200.
#     surround_distance_voxel = surround_distance_um / out_resolution_um
#     print "surround size (in voxels):", surround_distance_voxel

    # Reconstruct brain.

    reconstructed_brain = {}

    for name, cnts3d in contours_atlasResol.iteritems():
        reconstructed_brain[name] = interpolate_contours_to_volume(interpolation_direction='x',
                                                        contours_xyz=cnts3d, 
                                                        len_interval=20.,
                                                            return_origin_instead_of_bbox=True)

#         surround_name = convert_to_surround_name(name, margin='%dum' % surround_distance_um)

#         reconstructed_brain[surround_name] = \
#         get_surround_volume_v2(vol=reconstructed_brain[name][0], origin=reconstructed_brain[name][1], 
#                                wall_level=valid_level, distance=surround_distance_voxel, 
#                                prob=True,
#                                return_origin_instead_of_bbox=True)
    
    for s, v in reconstructed_brain.iteritems():
        vol_fp = DataManager.get_original_volume_filepath_v2(stack_spec=dict(name=stack, 
                                                                         vol_type='annotationAsScore',
                                                                resolution=out_resolution),
                                            
                                            structure=s)
        save_data(v[0], vol_fp, upload_s3=False)
        
        origin_fp = DataManager.get_original_volume_origin_filepath_v3(stack_spec=dict(name=stack, 
                                                                         vol_type='annotationAsScore',
                                                                        resolution=out_resolution),
                                            structure=s)
        save_data(v[1], origin_fp, upload_s3=False)

In [8]:
reconstructed_brain.keys()

[u'ExMC_L',
 u'MPul_L',
 u'LPul_L',
 u'IPul_L',
 u'InMC_L',
 u'ExPC_L',
 u'IPulCM_L',
 u'IPulM_L',
 u'InPC_L',
 u'IPulP_L',
 u'DLG_L',
 u'IPulCL_L',
 u'APul_L']